In [ ]:
import numpy as np
import seaborn as sns
from tqdm import trange
from scipy.stats import norm
from scipy.stats import uniform
import matplotlib.pyplot as plt

# plotting params
%matplotlib inline
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12
plt.rcParams['figure.figsize'] = (15.0, 8.0)
sns.set_style("white")

# path params
plot_dir = './plots/'

## 6.7 Importance Sampling for Off-Policy Estimation 

Recall that Monte Carlo sampling allows us to estimate the expectation of a random function by sampling from the corresponding probability distribution. More precisely, if p(x) is a 1-dim (continuous) probability density, and X ∼ p is a stochastic variable distributed according to this density p, then the expected value of some function f can be estimated using Monte Carlo
sampling by:

$$ E_p(f(X)) =\int f(x)p(x) dx \approx \frac{1}{n} \sum_{i=1}^{n}f(X_i)$$

for sample of independent $X_1, X_2,...,X_n ~p$.


## Importance Sampling

With importance sampling, we try to reduce the variance of our Monte-Carlo integral estimation by choosing a better distribution from which to simulate our random variables. It involves multiplying the integrand by 1 (usually dressed up in a “tricky fashion”) to yield an expectation of a quantity that varies less than the original integrand over the region of integration. Concretely,

$$\mathbb{E}_{p(x)} \big[\ f(x) \big] = \int f(x)\ p(x)\ dx = \int f(x)\ p(x)\ \frac{q(x)}{q(x)}\ dx = \int \frac{p(x)}{q(x)}\cdot f(x)\ q(x)\ dx = \mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big]$$

Thus, the MC estimation of the expectation becomes:

$$\mathbb{E}_{q(x)}  \big[\ f(x)\cdot \frac{p(x)}{q(x)} \big] \approx \frac{1}{N} \sum_{i=1}^{n} w_i \cdot f(x_i)$$

where $w_i = \dfrac{p(x_i)}{q(x_i)}$

1. Let X ∼ N(0; 1) be a standard normal stochastic variable. Use importance sampling to estimate $E(X^2)$ by sampling from a uniform distribution $q ∼ U(−5; 5)$ on the interval $[−5; 5]$. What value do you expect (based on your knowledge of the normal distribution)? How accurate is your estimate based on importance sampling?

In [ ]:
def Importance_Sampling_1(num_samples, plot=True):
    
    num_samples = int(num_samples)
    
    # parameters of the probability distributions
    mu_1, sigma_1 = 0, 1
    a, b = -5, 5
    
    # sample from the distributions
    Y = np.random.normal(mu_1, sigma_1, num_samples)
    X = np.random.uniform(a, b, num_samples)
    
    # setup pdfs
    norm_pdf = norm(mu_1, sigma_1)
    uniform_pdf = uniform(a, b-a)
    
    # calculate weight
    num = norm_pdf.pdf(X)
    denum = uniform_pdf.pdf(X)
    weights = num / denum
    
    # calculate X^2
    f_of_X = pow(X,2)
    f_of_Y = pow(Y,2)
    
    # calculate the expected value
    expected_value = (weights.dot(f_of_X))/num_samples
            
    # print resuts
    print("===================================================")
    print("Importance Sampling of size {}".format(num_samples))
    print("Expected value: {}".format(expected_value))
    print("===================================================\n")
    
    # weight the samples from the normal distribuiton
    IS_distribution = weights*f_of_X
    
    # plot and compare
    if plot:
        # histogram from uniform samples weighted by the normal distribution
        n, bins, patches = plt.hist(IS_distribution, 50, density=1, facecolor='green', alpha=0.75)
        plt.ylabel('Number of Samples')
        plt.xlabel(r'$w*X^2$, X sampled from uniform distribution')
        plt.title("Histogram, Importance Sampling")
        plt.grid(True)
        plt.savefig(plot_dir + 'uniform_sample_histogram{}.pdf'.format(num_samples), format='pdf', dpi=300)
        plt.show()
        
        # histogram from normal samples
        n, bins, patches = plt.hist(f_of_Y, 50, density=1, facecolor='blue', alpha=0.75)
        plt.ylabel('Number of Samples')
        plt.xlabel(r'$Y^2$, sampled from normal distribution')
        plt.title("Histogram")
        plt.grid(True)
        plt.savefig(plot_dir + 'normal_sample_histogram{}.pdf'.format(num_samples), format='pdf', dpi=300)
        plt.show()
        
        # weights 
        plt.scatter(f_of_X, weights)
        plt.ylabel('weights')
        plt.xlabel(r'$X^2$')
        plt.title("Weights values")
        plt.grid(True)
        plt.savefig(plot_dir + 'weights{}.pdf'.format(num_samples), format='pdf', dpi=300)
        plt.show()
        
    

In [ ]:
Importance_Sampling_1(1e5)

2. Suppose some random process produces output $(−1 ≤ X ≤ 1)$ that is distributed according to the following continuous density:
$$p(x) = \frac{1 + cos(\pi x)}{2}$$

Again we are interested in estimation $E(X^2)$. Use importance sampling to estimate this value. Quantify the uncertainty on your result.

In [ ]:
def Importance_Sampling_2(num_samples, plot=True):
    
    num_samples = int(num_samples)
    
    # parameters of the probability distributions
    a, b = -1, 1
    
    # sample from the distributions
    X = np.random.uniform(a, b, num_samples)
    
    # setup pdfs
    norm_pdf = norm(mu_1, sigma_1)
    uniform_pdf = uniform(a, b-a)
    
    # calculate weight
    num = (1+ np.cos(X))/2
    denum = uniform_pdf.pdf(X)
    weights = num / denum
    
    # calculate X^2
    f_of_X = pow(X,2)
    
    # calculate the expected value
    expected_value = (weights.dot(f_of_X))/num_samples

    # print resuts
    print("===================================================")
    print("Importance Sampling of size {}".format(num_samples))
    print("Expected value: {}".format(expected_value))
    print("===================================================\n")
    
    # weight the samples from the normal distribuiton
    IS_distribution = weights*f_of_X
    
    # plot and compare
    if plot:
        n, bins, patches = plt.hist(IS_distribution, 50, density=1, facecolor='green', alpha=0.75)
        plt.ylabel('Number of Samples')
        plt.xlabel(r'$X^2$, sampled from uniform distribution')
        plt.title("Histogram, Importance Sampling")
        plt.grid(True)
        plt.savefig(plot_dir + 'is_histogram_{}.pdf'.format(num_samples), format='pdf', dpi=300)
        plt.show()
        

In [ ]:
Importance_Sampling_2(1e6)

## References

* http://ib.berkeley.edu/labs/slatkin/eriq/classes/guest_lect/mc_lecture_notes.pdf